In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import glob
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import MinMaxScaler
import joblib
import os

In [2]:
dataset_path = "/Users/rajasekar.j/Projects/GaitAnalysis/dataset/normal_gait/"

file_list = glob.glob(dataset_path + "normal_*.csv")

# Load and combine all files
df_list = [pd.read_csv(file) for file in file_list]
combined_df = pd.concat(df_list, ignore_index=True)

# Features to use
features = [
    "XLeftShoulder", "YLeftShoulder", "ZLeftShoulder", "XRightShoulder", "YRightShoulder", "ZRightShoulder",
    "XLeftElbow", "YLeftElbow", "ZLeftElbow", "XRightElbow", "YRightElbow", "ZRightElbow",
    "XLeftWrist", "YLeftWrist", "ZLeftWrist", "XRightWrist", "YRightWrist", "ZRightWrist",
    "XLeftHip", "YLeftHip", "ZLeftHip", "XRightHip", "YRightHip", "ZRightHip",
    "XLeftKnee", "YLeftKnee", "ZLeftKnee", "XRightKnee", "YRightKnee", "ZRightKnee",
    "XLeftAnkle", "YLeftAnkle", "ZLeftAnkle", "XRightAnkle", "YRightAnkle", "ZRightAnkle",
    "ThetaLeftShoulder", "ThetaRightShoulder", "ThetaLeftHip", "ThetaRightHip",
    "ThetaLeftKnee", "ThetaRightKnee", "StepLength", "StepWidth", "FeetClearance",
    "LeftStrideSpeed", "RightStrideSpeed"
]

data = combined_df[features].values.astype(np.float32)

In [3]:
# Normalize data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Save the scaler
scaler_path = "/Users/rajasekar.j/Projects/GaitAnalysis/scaler.pkl"
joblib.dump(scaler, scaler_path)

# Define Autoencoder Model
input_dim = data_scaled.shape[1]
input_layer = Input(shape=(input_dim,))
encoded = Dense(64, activation='relu')(input_layer)
encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(16, activation='relu')(encoded)
decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(decoded)
output_layer = Dense(input_dim, activation='sigmoid')(decoded)

autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer='adam', loss='mse')

# Train Model
autoencoder.fit(data_scaled, data_scaled, epochs=50, batch_size=16, validation_split=0.2)

# Save Model to Google Drive
model_path = "/Users/rajasekar.j/Projects/GaitAnalysis/autoencoder_model.h5"
autoencoder.save(model_path)


Epoch 1/50
117254/117254 ━━━━━━━━━━━━━━━━━━━━ 68s 574us/step - loss: 0.0014 - val_loss: 4.4881e-04
Epoch 2/50
117254/117254 ━━━━━━━━━━━━━━━━━━━━ 68s 577us/step - loss: 4.8716e-04 - val_loss: 3.8738e-04
Epoch 3/50
117254/117254 ━━━━━━━━━━━━━━━━━━━━ 67s 574us/step - loss: 4.3085e-04 - val_loss: 3.5319e-04
Epoch 4/50
117254/117254 ━━━━━━━━━━━━━━━━━━━━ 67s 572us/step - loss: 4.0317e-04 - val_loss: 3.5265e-04
Epoch 5/50
117254/117254 ━━━━━━━━━━━━━━━━━━━━ 68s 579us/step - loss: 3.8909e-04 - val_loss: 3.5777e-04
Epoch 6/50
117254/117254 ━━━━━━━━━━━━━━━━━━━━ 66s 559us/step - loss: 3.8220e-04 - val_loss: 3.4093e-04
Epoch 7/50
117254/117254 ━━━━━━━━━━━━━━━━━━━━ 65s 558us/step - loss: 3.7596e-04 - val_loss: 3.3915e-04
Epoch 8/50
117254/117254 ━━━━━━━━━━━━━━━━━━━━ 66s 559us/step - loss: 3.7111e-04 - val_loss: 3.4712e-04
Epoch 9/50
117254/117254 ━━━━━━━━━━━━━━━━━━━━ 66s 560us/step - loss: 3.6757e-04 - val_loss: 3.4667e-04
Epoch 10/50
117254/117254 ━━━━━━━━━━━━━━━━━━━━ 215s 2ms/step - loss: 3.6404e-